# Day1 強化学習の位置付けを知る

## 準備

以下の Python ライブラリを使用します．

- `gym`
- `numpy`

## 環境とは？

強化学習では，「環境」が与えられます．この環境に対する，より良い行動パターンを見つけ出すことが強化学習の目的です．

> 環境とは「行動」と行動に応じた「状態」の変化が定義されており，ある状態への到達に対し「報酬」が与えられる空間のことです．端的には，ゲームのようなものです．ゲームでは，ボタンを押したらキャラクターがジャンプしたりします．「ボタンを押す」のが行動であり，「キャラクターがジャンプする」のが状態の変化に相当します．そしてゴールに到達できれば「報酬」が得られます． *( p.15 )*

ここでは，**FrozenLake-v0**（以下，FL）というテキストゲームを例にとって「環境」とは何かを説明します．

### FrozenLake-v0 について

FL の説明は[ココ](https://gym.openai.com/envs/FrozenLake-v0/)に乗っています．リンク先には実際のプレイ動画もあります．「凍った湖の上に落としてしまったフリスビーを取りに行く」という筋書きらしいです．湖は次のような４ｘ４のマップで表されます．

```
SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)
```

スタート地点である `S` のマスから出発して，ゴール地点（フリスビーのある場所）である `G`のマスに辿り着けばゲームグリアです．ただし，所々に溶けて穴が空いているマス `H` があり，ここに落ちてしまうとゲームオーバーになります．移動方法は「上（下・左・右）に進む」の４つです．しかし，`F` のマスは凍っていてよく滑るので，望んだ方向に進める確率は１/３です．例えば，上に進もうとした場合逆方向である下以外（左・右）にも同じ確率で進む可能性があります．

### FrozenLake-v0 で遊ぶ

OpenAI が **gym** という Python ライブラリを公開しています．gym には強化学習に使える様々な環境が用意されていて，その中に FL もあります．これをこのノート用に扱いやすくしたラッパークラスを `day1/frozen_lake_wrap.py` に用意しました．

In [1]:
from frozen_lake_wrap import FrozenLake, Action

`FrozenLake` がゲーム本体，`Action` は列挙型で移動方法が定義されています．実際に動かしてみましょう．

In [2]:
env = FrozenLake(is_slippery=False)
env.reset()
env.env.render()
env.step(Action.DOWN)
env.env.render()
env.close()


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG


コンストラクタで `is_slippery=False` を指定すると，`F` のマスで滑らなくなる，つまり望んだ方向に必ず進めるようになります．FrozenLake のメソッドをいくつか紹介します．

- `env.reset()` ゲームを初期化する
- `env.step()` １マス移動する
- `env.render()` 盤面を描画する
- `env.close()` ゲームを終了する

`step()` には `Action.UP`,`Action.DOWN`,`Action.RIGHT`,`Action.LEFT` の内１つを指定します．上記のコードでは `Action.DOWN` を指定したので，スタート地点 $(0,0)$ から $(1,0)$ に移動したわけです．ここで，位置座標は（行，列）としていますから，$(1,0)$ は１行目の０列目のことです．

### 環境 ＝ マルコフ決定過程

強化学習における環境とは，**マルコフ決定過程（Markov Decision Process: MDP）**のことです．マルコフ決定過程は，４つの要素の組 $ \langle S, A, T, R \rangle $ で表されます．$S$ は状態集合，$A$ は行動集合，$T$ は遷移関数，$R$ は報酬関数です．

#### 状態集合

FL において，「状態」とは位置座標のことです．このゲームは４×４マスなので，状態は全部で 16 通りあるわけです．状態集合 $S$ とはこれら全ての状態を要素とする集合を指します．

$$ S = \left\{ (i,j) \mid 0 \leq i,j < 4 \right\} $$

#### 行動集合

FL の「行動」は「上（下・左・右）への移動」です．これをそれぞれ $a_U$，$a_D$，$a_L$，$a_R$ とすると，行動集合 $A$ は次のようになります．

$$ A = \left\{ a_U, a_D, a_L, a_R \right\} $$

#### 遷移関数

遷移関数は，状態 $s$ で行動 $a$ を取ったとき状態 $s'$ に遷移する確率を表します．

$$T : S \times A \times S \rightarrow [0,1];\hspace{4pt} (s,a,s') \mapsto T(s' \mid s,a)$$

FL は望んだ方向に進める確率が１/３でした．例えば $(2,2)$ から上に移動しようとした場合，$(1,2)$ に移動できる確率は１/３ですから，このとき遷移関数の値は

$$ T(s'=(1,2) \mid s=(2,2), a=a_U) = \frac{1}{3} $$

となります．一方，逆方向 $(3,2)$ に移動することはありませんから，このときの遷移関数の値は $0$ です．

$$ T(s'=(3,2) \mid s=(2,2), a=a_U) = 0 $$

#### 報酬関数

マルコフ決定過程では，「状態 $s$ で行動 $a$ を取り状態 $s'$ に遷移する」たびに「報酬」が貰えます．この報酬を与えるのが報酬関数です．

$$ R : S \times A \times S \rightarrow \mathbb{R};\hspace{4pt} (s,a,s') \mapsto R(s,a,s') $$

FL ではゴールのマス $(3,3)$ に移動すると報酬として $1$ が与えられます．

$$ R(s,a,s'=(3,3)) = 1$$

一方，それ以外のマスに移動したときの報酬は $0$ です．

$$ R(s,a,s'\neq(3,3)) = 0 $$

このように，FL の報酬関数 $R$ は遷移先 $s'$ にのみ依存します．強化学習はこの報酬を目安にして学習を行うわけです．

### 環境としての FrozenLake-v0



`FrozenLake` の `step()` には３つの返り値があります．

In [18]:
env = FrozenLake(is_slippery=True)
env.reset()
pos, reward, done = env.step(Action.DOWN)
print('pos:%s, reward:%s, done:%s'%(pos, reward, done))

pos:(0, 1), reward:0.0, done:False


`pos` は遷移先，`reward` は報酬，`done` はゴールしたかどうかを示すフラグです．遷移先は FL の遷移関数に従って確率的に決定され，報酬は報酬関数によって与えられます．少し遷移関数を見てみましょう．`FrozenLake` では，遷移関数は (４×４)×４×(４×４) の numpy 配列で表されています（遷移関数 $T$ の定義域は $S \times A \times S$ です）．例えば $(2,2)$ から上に移動しようとしたときの遷移確率行列は次のようになります．

In [10]:
print(env.T[(2,2)][Action.UP])

[[0.         0.         0.         0.        ]
 [0.         0.         0.33333333 0.        ]
 [0.         0.33333333 0.         0.33333333]
 [0.         0.         0.         0.        ]]


上 $(1,2)$，左 $(2,1)$，右 $(2,3)$ に移動する確率がそれぞれ $\frac{1}{3}$，それ以外に移動する確率が $0$ となっていることが分かります．$(0,0)$ から左に移動しようとした場合は少し特殊です．

In [13]:
print(env.T[(0,0)][Action.LEFT])

[[0.66666667 0.         0.         0.        ]
 [0.33333333 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


$(0,0)$ に遷移する確率が $\frac{2}{3}$ となっています．FL では，遷移先がマップ外になる場合，元の位置が遷移先となります．$(0,0)$ の左側 $(0,-1)$ はマップの外であり，また上側 $(-1,0)$ もまたマップ外ですから，左に移動する場合と上に移動する場合，遷移先は共に $(0,0)$ になります．なので，合わせて$\frac{2}{3}$ の確率で $(0,0)$ に遷移する，という計算になるわけです．

次は報酬関数を見てみましょう．報酬関数は４×４の numpy 配列です．

In [11]:
print(env.R)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]


ゴール地点 $(3,3)$ での報酬が $1$，それ以外の場所では $0$ となっていることが分かります．それでは最後に，`FrozenLake` が強化学習のための環境（マルコフ決定過程）であることを実際に動かしながら確かめてみましょう．以下のセルを実行すると，ランダムな行動を取りながらゲームが進み，その度に遷移先に応じた報酬が貰える様子が見れます．また，その遷移が遷移関数に従っていることも分かるはずです（`is_slippery=True` としているので，望んだ方向に進む確率は $1$ になります）．

In [39]:
from IPython.display import clear_output
from time import sleep

def render(env, state, action, new_state, reward, done):
    if not done:
        print('---------------- Now Playing... ----------------')
    elif env.map[new_state[0]][new_state[1]]=='G':
        print('----------------- Game Clear! ------------------')
    else:
        print('----------------- Game Over... -----------------')
    
    env.render()
    print('------------------------------------------------')
    print('action:%s at %s,'%(action, state))
    print('new_state:%s, reward:%s, done:%s,'%(new_state, reward, done))
    if state is not None and action is not None:
        print('T(s\'|s=%s, a=%s):\n%s'%(state, action, env.T[state][action]))

env.close()
env = FrozenLake(is_slippery=False)
state = env.reset()
done = False
delay = 5

render(env, state, None, None, 0, False)
while not done:
    sleep(delay)
    action = Action.choice()
    new_state, reward, done = env.step(action)
    clear_output(wait=True)
    render(env, state, action, new_state, reward, done)
    state = new_state

env.close()

----------------- Game Over... -----------------
  (Right)
SFFF
FHFH
FFFH
HFFG
------------------------------------------------
action:Action.RIGHT at (1, 0),
new_state:(1, 1), reward:0.0, done:True,
T(s'|s=(1, 0), a=Action.RIGHT):
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
